# ENGRI 1120 Lab 9: Computation of an Equilibrium Constant

### Introduction
One potential strategy to generate clean hydrogen gas $H_{2}(g)$ for use in applications such as transportation is to directly decompose ammonia NH$_{3}(g)$ in the gas phase reaction:

$$2NH_{3}(g)~{\rightleftharpoons}~N_{2}(g)+3H_{2}(g)$$

Let's explore the equilibrium behavior of this reaction in a constant volume well-mixed _batch_ reactor (no input or output streams) with $V$ = 100 L operating at T = 900$^{\circ}$C and an unknown pressure (measured in units of MPa). 

Initially, there are 100 mol of NH$_{3}(g)$ and no products in the reactor. The teaching team measured the equilibrium extent of reaction $\epsilon_{1}^{\star}$ = 34.14 mol.

__Assumptions__
* The gas phase can be treated as ideal
* The ideal gas law can be used to describe the conditions in the reactor
* The reference pressure is $P^{\circ}$ = 100 MPa
* R = 0.008314 L MPa mol$^{-1}$ K$^{-1}$ and R = 0.008314 kJ mol$^{-1}$ K$^{-1}$ 

### Lab setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-9-fun-w-equilibrium`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-9-fun-w-equilibrium/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-9-fun-w-equilibrium/Manifest.toml`


In [8]:
# load req packages -
using PrettyTables

In [11]:
# Setup the problem -
T = (900 + 273.15); # units: K
V = 100.0; 			# units: L
R = 0.008314 		# units: kJ/mol-K
R_MPa = 0.008314 	# units: L MPa mol^-1 K^-1
Pₒ = 100.0 			# units: MPa
ϵₑ = 34.14;         # units: mol (given in the problem)

# stoichiometry -
S = [
    -2.0 ; # 1 NH3
    1.0  ; # 2 N2
    3.0  ; # 3 H2
];

# setup initial -
n_initial = [
    100.0 	; 	# 1 NH3
    0.0 	; 	# 2 N2
    0.0 	; 	# 3 H2
]

# we looked these up ...
G_array = [
    -16.4 	; 	# 1 NH3 kJ/mol
    0.0 	; 	# 2 N2 kJ/mol
    0.0 	; 	# 3 H2 kJ/mol
];

# ΔG formation -
ΔG_formation = sum(G_array.*S);

### a) Compute the system state table
To compute the missing values in the state table, use the equilibrium extent $\epsilon_{1}^{\star}$ and the mole balances to compute the number of mole of component $i$ at equilibrium:

$$n_{i} = n_{i}^{\circ}+\sigma_{i1}\epsilon_{1}^{\star}\qquad{i=1,2,\dots,\mathcal{M}}$$

Then compute the mole fractions that are required to estimate $K$. 

In [12]:
# compute the final number of mols -
n_final_eq = n_initial + S*ϵₑ

# compute the mol fraction -
n_total = sum(n_final_eq);

# compute the mol fraction -
y_final_eq = (1/n_total)*n_final_eq

# header row -
state_table_header_row = (["Species","σ_i1","nᵢ_initial","nᵢ_final","yᵢ_final"],["","","mol", "mol", ""]);

# setup state table data array =
state_table_array = Array{Any,2}(undef, 4,5)

# NH3 -
state_table_array[1,1] = "NH₃"
state_table_array[1,2] = S[1]
state_table_array[1,3] = n_initial[1]
state_table_array[1,4] = n_final_eq[1]
state_table_array[1,5] = y_final_eq[1]

# N2 -
state_table_array[2,1] = "N₂"
state_table_array[2,2] = S[2]
state_table_array[2,3] = n_initial[2]
state_table_array[2,4] = n_final_eq[2]
state_table_array[2,5] = y_final_eq[2]

# H2 -
state_table_array[3,1] = "H₂"
state_table_array[3,2] = S[3]
state_table_array[3,3] = n_initial[3]
state_table_array[3,4] = n_final_eq[3]
state_table_array[3,5] = y_final_eq[3]

# total -
state_table_array[4,1] = "Total"
state_table_array[4,2] = sum(S)
state_table_array[4,3] = sum(n_initial)
state_table_array[4,4] = sum(n_final_eq)
state_table_array[4,5] = sum(y_final_eq)

# write the table -
pretty_table(state_table_array; header=state_table_header_row)

┌─────────┬──────┬────────────┬──────────┬──────────┐
│ Species │ σ_i1 │ nᵢ_initial │ nᵢ_final │ yᵢ_final │
│         │      │        mol │      mol │          │
├─────────┼──────┼────────────┼──────────┼──────────┤
│     NH₃ │ -2.0 │      100.0 │    31.72 │ 0.188495 │
│      N₂ │  1.0 │        0.0 │    34.14 │ 0.202876 │
│      H₂ │  3.0 │        0.0 │   102.42 │ 0.608628 │
│   Total │  2.0 │      100.0 │   168.28 │      1.0 │
└─────────┴──────┴────────────┴──────────┴──────────┘


### b) Compute the missing value for the pressure $P$ in the reactor (units: MPa)

In [13]:
# We have the total number of moles from the state table, use the IG equation of state to estimate 
# pressure (note: this is an insanely bad idea ... but we do it anyway)
P_eq = (n_total)*(R_MPa*T)*(1/V)

16.413306081480002

### c) Compute a value for the gas-phase ideal equilibrium constant

In [18]:
# compute the K-value from the extent -
P_term = (P_eq/Pₒ)^2
K₁ = (y_final_eq[1]^S[1])*(y_final_eq[2]^S[2])*(y_final_eq[3]^S[3])
K₂ = P_term*K₁

println("The Keq = $(K₂) from the extent of reaction")

The Keq = 0.034679961303443675 from the extent of reaction


In [17]:
# check: we can also compute the Keq from the ΔG expression -
K_problem = exp(-1*ΔG_formation/(R*T));
println("Check: Keq = $(K_problem) from the KNOWN ΔG at T = $(T) K")

Check: K_eq = 0.03463565811755417 from the KNOWN ΔG at T = 1173.15 K


### d) Compute the $\Delta{G}^{\circ}$ of reaction for the decomposition of NH$_{3}$(g). Is this reaction energetically favorable (yes -or- no)?

In [20]:
ΔG_problem = -R*T*log(K₂);
println("Computed ΔG = $(ΔG_problem) kJ mol^{-1} K^{-1} > 0 => NOT favorable")

Computed ΔG = 32.787531980601926 kJ mol^{-1} K^{-1} > 0 => NOT favorable
